In [3]:
import pandas as pd
import datetime
from datetime import datetime
import regex as re
import numpy as np
import os

In [4]:
small_files_path = r'random_songs_split_after_crawling'

In [5]:
# pattern of date and views as taken from youtube:
pattern = re.compile("^([A-Z][a-z]{1}[A-z])[ ](0?[1-9]|[12][0-9]|3[01])[ ]\d{4}$")

In [6]:
# concats youtube and google trends crawling data from many small CSVs.
def union_csvs(path):
    results_df = pd.DataFrame()
    for filename in os.listdir(path):
        if filename.endswith(".csv") and not filename.startswith('combinedfile'):
            file_df = pd.read_csv(path+'\\' +filename)
            results_df = results_df.append(file_df)
    return results_df.drop_duplicates(subset='track_id')

# normalizes 
def normalize(df):
#     df['duration_ms_stand'] = np.where(df['duration_ms'] >= 10 * 60 * 1000, 10 * 60 * 1000, df['duration_ms'])
    df = df[df['duration_ms'] <= 10 * 60 * 1000]
    df['duration_norm'] = (df['duration_ms']-df['duration_ms'].min())/(df['duration_ms'].max()-df['duration_ms'].min())
    
    df['loudness_stand'] = np.where(df['loudness'] <= -40, -40 , df['loudness'])
    df['loudness_norm'] = (df['loudness_stand']-df['loudness_stand'].min())/(df['loudness_stand'].max()-df['loudness_stand'].min())

    df['tempo_norm'] = (df['tempo']-df['tempo'].min())/(df['tempo'].max()-df['tempo'].min())
    return df
    
# calculates the number of days passed since the clip was uploaded to youtube
def days(strdate):
    strdate = strdate.lstrip()
    date = datetime.strptime(strdate, '%b %d %Y')
    date_oct = datetime.strptime('Oct 1 2018', '%b %d %Y')

    if date_oct <= date:
        return -1

    return (date.today() - date).days

# extracts the number of views and number of days since file was uploaded
def extract_dates_and_views(data):
    print(str(len(data)) + ' before filtering') 

    data = data[data['date_and_views'] != "(None, None)"]
    print(str(len(data)) + ' after removing rows when views and dates are both None') 
    data['str_dv'] = data['date_and_views'].astype(str)
    data['str_dv'] = data.str_dv.apply(lambda x: x.replace('(','').replace(')','',).replace(',','-',1).split('-'))
    data['views'] = data.str_dv.apply(lambda x: x[0].replace("'", ''))
    data = data[data['views'].apply(lambda x: x.isdigit())]
    print(str(len(data)) + ' after removing rows when views is not a digit') 
    data['views'] = data.views.apply(lambda x: int(x))
    data['upload_date'] = data.str_dv.apply(lambda x: x[1].replace(',','').replace('on','').replace("'",'').lstrip())
    data = data[data['upload_date'] != "None"]
    print(str(len(data)) + ' after removing rows where date is None') 
    data = data[data.upload_date.apply(lambda s: pattern.match(s) != None)]
    print(str(len(data)) + ' after removing rows where pattern does not match') 
    data['days_since_upload'] = data.upload_date.apply(lambda x: days(x))
    data = data[data['days_since_upload'] >0]
    print(str(len(data)) + ' after removing rows where song is too new') 
    return data.drop('str_dv',axis=1)

def add_popularity_measures(data):
    data['days_views_ratio'] = data['views']/ data['days_since_upload']
    data['days_views_ratio'] = np.where(data['days_views_ratio'] >= 10000, 10000, data['days_views_ratio'])
    data['youtube_popularity'] = np.ceil(((data['days_views_ratio'] - (data['days_views_ratio'].min())) / \
                             (data['days_views_ratio'].max() - data['days_views_ratio'].min()))*100)
    data['spotify_popularity'] = np.where(data['popularity'] == 0, 1, data['popularity'])
    data['youtube_spotify_popularity_ratio'] = data['youtube_popularity']/ data['spotify_popularity']
    return data

csvs were split to small files with 100 rows each in order to perform the crawling. the first thing that we do is concat all of them:

In [7]:
raw_data = union_csvs(small_files_path).drop(labels = ['Unnamed: 0','Unnamed: 0.1'],axis =1 )
raw_data.sample(3)

,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,...,Vatican City,Venezuela,Vietnam,Wallis & Futuna,Western Sahara,Yemen,Zambia,Zimbabwe,Åland Islands,date_and_views
25,Tray Loop,5UGZizzQ5jwuucU36aXrF9,Fuck You,0.168,0.633,206856,0.627,0.0347,9,0.0887,...,0,0,5,0,0,0,0,0,0,"('2216304', 'Dec 21, 2016')"
10,DragonFlex,1Y1jVQKIoLuKoWgUckIVMI,Flags,0.995,0.438,80640,0.137,0.8920,6,0.1140,...,0,0,1,0,0,0,0,0,0,"('751495', 'Sep 1, 2016')"
82,Adrian Von Ziegler,2xKqGMDMf0XeO6Amivi3CF,Till Valhöll,0.529,0.363,287250,0.213,0.9150,8,0.1200,...,0,0,0,0,0,0,0,0,0,"('417381', 'Aug 9, 2015')"


some coloumns need to be normalized between 0 to 1, and songs longer than 10 minutes are removed (mostly classical music or podcasts):

In [8]:
normalized = normalize(raw_data)
normalized.head(3)

c:\users\netta\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\netta\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
c:\users\netta\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,...,Western Sahara,Yemen,Zambia,Zimbabwe,Åland Islands,date_and_views,duration_norm,loudness_stand,loudness_norm,tempo_norm
0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",0.00582,0.743,238373,0.339,0.0,1,0.0812,...,0,0,0,0,0,"(156534403, Jun 23, 2018)",0.393517,-7.678,0.792109,0.815763
1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),0.02440,0.846,214800,0.557,0.0,8,0.2860,...,0,0,0,0,0,"(27490037, Jul 26, 2018)",0.353910,-7.259,0.802377,0.636079
2,R3HAB,6Wosx2euFPMT14UXiWudMy,Radio Silence,0.02500,0.603,138913,0.723,0.0,9,0.0824,...,0,0,0,0,0,"(619066, Aug 3, 2018)",0.226408,-5.890,0.835927,0.459895


youtube upload dates and number of views are extrcted, and filtering is done for invalid values:

In [9]:
filtered = extract_dates_and_views(normalized)
filtered.head(3)

42138 before filtering
40742 after removing rows when views and dates are both None


c:\users\netta\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\netta\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\netta\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

40452 after removing rows when views is not a digit
40452 after removing rows where date is None
40278 after removing rows where pattern does not match
37234 after removing rows where song is too new


,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,...,Zimbabwe,Åland Islands,date_and_views,duration_norm,loudness_stand,loudness_norm,tempo_norm,views,upload_date,days_since_upload
0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",0.00582,0.743,238373,0.339,0.0,1,0.0812,...,0,0,"(156534403, Jun 23, 2018)",0.393517,-7.678,0.792109,0.815763,156534403,Jun 23 2018,253
1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),0.02440,0.846,214800,0.557,0.0,8,0.2860,...,0,0,"(27490037, Jul 26, 2018)",0.353910,-7.259,0.802377,0.636079,27490037,Jul 26 2018,220
2,R3HAB,6Wosx2euFPMT14UXiWudMy,Radio Silence,0.02500,0.603,138913,0.723,0.0,9,0.0824,...,0,0,"(619066, Aug 3, 2018)",0.226408,-5.890,0.835927,0.459895,619066,Aug 3 2018,212


popularity measures are added according to the ratio of views and date since upload. the long tail is cut and the ration is normalized:

In [10]:
with_popularity = add_popularity_measures(filtered)
with_popularity.head(3)

,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,...,loudness_stand,loudness_norm,tempo_norm,views,upload_date,days_since_upload,days_views_ratio,youtube_popularity,spotify_popularity,youtube_spotify_popularity_ratio
0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",0.00582,0.743,238373,0.339,0.0,1,0.0812,...,-7.678,0.792109,0.815763,156534403,Jun 23 2018,253,10000.000000,100.0,44,2.272727
1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),0.02440,0.846,214800,0.557,0.0,8,0.2860,...,-7.259,0.802377,0.636079,27490037,Jul 26 2018,220,10000.000000,100.0,10,10.000000
2,R3HAB,6Wosx2euFPMT14UXiWudMy,Radio Silence,0.02500,0.603,138913,0.723,0.0,9,0.0824,...,-5.890,0.835927,0.459895,619066,Aug 3 2018,212,2920.122642,30.0,63,0.476190


redundant columns are removed and csv is exported:

In [11]:
with_popularity.artist_popularity = with_popularity.artist_popularity/100
with_popularity.to_csv("random_songs_processed_dataset.csv",encoding='utf-8',index = False)